In [1]:
import pickle
with open('../pickles/data_cleaned.pickle', 'rb') as handle:
    data_cleaned = pickle.load(handle)

FileNotFoundError: [Errno 2] No such file or directory: '../pickles/data_cleaned.pickle'

In [2]:
data_cleaned.keys()

dict_keys(['RS_2020_confession', 'confession_comments', 'RS_2020_confessions', 'confessions_comments', 'RS_2020_creepypasta', 'creepypasta_comments', 'RS_2020_nosleep', 'nosleep_comments', 'RS_2020_self', 'self_comments', 'RS_2020_shortscarystories', 'shortscarystories_comments', 'RS_2020_storiesaboutkevin', 'storiesaboutkevin_comments', 'RS_2020_tifu', 'tifu_comments'])

In [3]:
for k,v in data_cleaned.items():
    print(k)
    print('Number of posts: ', len(v.index), '\n')

RS_2020_confession
Number of posts:  206075 

confession_comments
Number of posts:  1315746 

RS_2020_confessions
Number of posts:  22863 

confessions_comments
Number of posts:  711770 

RS_2020_creepypasta
Number of posts:  2391 

creepypasta_comments
Number of posts:  61105 

RS_2020_nosleep
Number of posts:  15426 

nosleep_comments
Number of posts:  1378102 

RS_2020_self
Number of posts:  17600 

self_comments
Number of posts:  818179 

RS_2020_shortscarystories
Number of posts:  54974 

shortscarystories_comments
Number of posts:  191192 

RS_2020_storiesaboutkevin
Number of posts:  3403 

storiesaboutkevin_comments
Number of posts:  21343 

RS_2020_tifu
Number of posts:  17135 

tifu_comments
Number of posts:  2795480 



In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
data_cleaned['RS_2020_storiesaboutkevin']['id'] = 't3_' + data_cleaned['RS_2020_storiesaboutkevin']['id'].astype(str)

In [6]:
data_cleaned['storiesaboutkevin_comments']

,parent_id,score,body,link_id,body_language
0,t3_h17ue6,1,See if your Kevin can out-stare a cat. He'll l...,t3_h17ue6,en
3,t3_h17ue6,1,Wait how did he have a staring contest if the ...,t3_h17ue6,en
7,t3_gz9cqx,1,Please correct me if I'm mistaken but I think ...,t3_gz9cqx,en
11,t3_gz9cqx,1,Suddenly I am reminded of the character from A...,t3_gz9cqx,en
15,t3_h7bbs5,1,Absolute gold! 🏅,t3_h7bbs5,en
...,...,...,...,...,...
69811,t3_2ht4oe,37,"Sorry, but if you are self aware enough to won...",t3_2ht4oe,en
69812,t3_2emlgx,14,I'm not retarded. At least... I hope not.,t3_2emlgx,en
69817,t3_2bptpe,4,Same here...,t3_2bptpe,en
69818,t3_2a02qe,13,"i think it was definitely a waste of time, but...",t3_2a02qe,en


In [7]:
x = pd.merge(data_cleaned['RS_2020_storiesaboutkevin'], data_cleaned['storiesaboutkevin_comments'], how = 'inner', left_on = 'id', right_on = 'parent_id')
x.groupby(['title', 'body']).first()

NameError: name 'pd' is not defined

In [8]:
test_text = data_cleaned['RS_2020_storiesaboutkevin'].iloc[0]['selftext']

In [9]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [10]:
doc = nlp(test_text)

from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np

embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

x = [sent.text for sent in doc.sents]
print(len(x))
data = embedder.encode(x)
data = data.reshape((10,768,1))
target = np.expand_dims(np.ones(10), axis = 1)

10


In [11]:
data.shape[1:]

(768, 1)

In [15]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout


In [13]:
model = Sequential()

model.add(LSTM(128, input_shape=data.shape[1:], return_sequences = True))
model.add(Dropout(0.1))

model.add(LSTM(128))
model.add(Dropout(0.1, name = 'layer'))

model.add(Dense(1, activation = 'sigmoid'))

In [16]:
model.compile(loss = 'binary_crossentropy',
             optimizer = keras.optimizers.Adam(lr=0.000959, beta_1 = 0.9, beta_2=0.999), # you can tune the learning rate here. Default lr = 0.01
             metrics=['accuracy'])

In [19]:
intermediate_layer_model = keras.Model(inputs=model.input,
                                 outputs=model.get_layer('layer').output)
intermediate_output = intermediate_layer_model.predict(data)

intermediate_output

In [155]:
data

array([[[ 0.6356785 ],
        [-0.33482894],
        [-0.20639172],
        ...,
        [ 0.05474838],
        [-0.9565444 ],
        [-0.5991295 ]],

       [[ 0.60855883],
        [-0.06837752],
        [ 0.4189801 ],
        ...,
        [ 0.23462464],
        [-0.540445  ],
        [-0.15702581]],

       [[-0.0566089 ],
        [-0.2259286 ],
        [ 0.23525931],
        ...,
        [ 0.21601215],
        [-0.43980205],
        [-0.8149141 ]],

       ...,

       [[ 0.15632093],
        [-0.5775589 ],
        [-0.01583201],
        ...,
        [ 0.4154871 ],
        [-0.5812536 ],
        [ 0.10089789]],

       [[ 0.45729637],
        [ 0.2174945 ],
        [ 0.3085463 ],
        ...,
        [ 0.01471978],
        [ 0.45252237],
        [ 0.42063355]],

       [[ 0.47682437],
        [ 0.59148335],
        [ 0.4708005 ],
        ...,
        [-0.60607916],
        [ 0.09449449],
        [-0.47713733]]], dtype=float32)